In [1]:
import kagglehub
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline

# Download latest version
#path = kagglehub.dataset_download("miguelaenlle/massive-stock-news-analysis-db-for-nlpbacktests")

2025-02-07 19:34:30.417331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738974870.426710    3714 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738974870.429659    3714 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-07 19:34:30.439676: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#Check how many NVDA articles there are in a specific dataset.
df = pd.read_csv("~/projects/NVDA_stock_prediction/Data/NewsArticles/raw_partner_headlines.csv")
len(df[df['stock']=='NVDA'])

0

In [54]:
def annotate_dataset_with_sentiment(dataset_path, model_name, output_path=None):
    """
    Load a dataset with a 'headline' and 'date' column, runs sentiment analysis
    using a pretrained BERT model on the headlines, and appends the sentiment to the dataset.

    Args:
        dataset_path (str): Path to the dataset file (e.g., CSV).
        model_name (str): Pretrained model identifier or local model path.
        output_path (str): Optional path to save the annotated dataset.

    Returns:
        pd.DataFrame: The dataset with a new 'sentiment' column.
    """
    # Load the dataset
    df = pd.read_csv(dataset_path)
    df = df[df['stock'] == 'NVDA']
    
    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
    
    # Create a sentiment analysis pipeline
    sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
    
    #Change predictions to sentiments with SENTIMENT_MAP dictionary.
    SENTIMENT_MAP = {'LABEL_0': 'negative', 'LABEL_1': 'neutral', 'LABEL_2': 'positive'}
    
    labels = []
    count = 0
    # Process each headline using the sentiment pipeline
    for headline in df['headline']:
        count += 1
        print(count)
        result = sentiment_pipeline(headline)
        
        #Result is a list with one dict in the format of [{'label': 'LABEL_1', 'score': 0.6225356459617615}]
        prediction = result[0]['label']
        labels.append(prediction)
    
    # Append sentiment to the DataFrame
    df['labels'] = labels
    sentiments= [SENTIMENT_MAP[label] for label in df['labels']]
    df['sentiments'] = sentiments
    
    # Optionally, save the annotated DataFrame
    if output_path:
        df.to_csv(output_path, index=False)
    
    return df


In [ ]:
annotated_df = annotate_dataset_with_sentiment("~/projects/NVDA_stock_prediction/Data/NewsArticles/raw_analyst_ratings.csv", 
                                               "/home/tommytang111/projects/NVDA_stock_prediction/Models/BERT_stock_news_sentiment", '/home/tommytang111/projects/NVDA_stock_prediction/Data/Outputs/raw_analyst_ratings_sentiment.csv')


Some layers from the model checkpoint at /home/tommytang111/projects/NVDA_stock_prediction/Models/BERT_stock_news_sentiment were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /home/tommytang111/projects/NVDA_stock_prediction/Models/BERT_stock_news_sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceCl